## 1.1-Importando librerias necesarias.

In [1]:
from gensim.models import KeyedVectors
import matplotlib.pyplot as  plt
from collections import Counter
import pandas as pd
import numpy as np
import random
import string
import math
import re
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

/home/fedricio/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/fedricio/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 1-Dataset Puestos

In [2]:
df_Jobs = pd.read_csv("Archivos 2-Especificos/Ejemplo1_Dataset_CVs_And_Job_Desc/EN/Job_Descr/Job_Positions_Scrapped/10_Positions_Only_Qualif.csv")
df_Jobs = df_Jobs.rename(columns={'job_title': 'Job_Title', 'job_description':'Job_Description'})

df_Jobs = df_Jobs.sort_values('Job_Title', ascending=True)
df_Jobs

,Job_Title,Job_Description
2,Data Scientist,"Master’s degree or above in a STEM field, incl..."
3,Data Scientist 2,"\nReporting to the Director, Data & Analytics,..."
6,HCM Consultant,\nThe Oracle Cloud HCM Absence Consultant will...
7,HCM Consultant 2,4+ years of experience in PeopleSoft or Oracle...
0,Machine Learning Engineer,Leveraging the latest machine and deep learnin...
1,Machine Learning Engineer 2,Collaborate with a multidisciplinary team to g...
8,Security Specialist,Work in a fast-paced environment that combine ...
9,Security Specialist 2,\n Handling incoming requests for assistanc...
4,Web Developer Full Stack,\n\n Graduate Degree in Information Technol...
5,Web Developer Full Stack 2,\n· Enter existing website codebases and exten...


### 2-Dataset CVs

### PDF a texto usando pdfplumber.

In [3]:
import pdfplumber
import os
import collections
from os import listdir
from os.path import isfile, join

#Leemos los CVs almacenados en nuestra carpeta y los extraemos uno por uno convirtiendolos a texto
#mediante la libreria pdfplumber:
pathCVs='Archivos 2-Especificos/Ejemplo1_Dataset_CVs_And_Job_Desc/EN/PDFs_CVs_2'
onlyfiles = [os.path.join(pathCVs, f) for f in os.listdir(pathCVs) if os.path.isfile(os.path.join(pathCVs, f))]
print("Cantidad de CVs extraidos:", len(onlyfiles))

#Funcion para extraer las palabras del CV:

def pdfextract(PDF_file):
    single_page_text = ""
    all_text = ""
    pdf = pdfplumber.open(PDF_file)
    for pdf_page in pdf.pages:
        single_page_text = pdf_page.extract_text()
        all_text = all_text + '\n' + single_page_text
    pdf.close()
    #return(all_text)        
    return(all_text.encode('utf-8'))

def extract_text(file):
    text = pdfextract(file).decode('utf-8')
    #text = pdfextract(file)
    return text

Cantidad de CVs extraidos: 10


In [4]:
#Obtenemos todas las palabras del CV sin preprocesar ni nada:
df_Candidates=pd.DataFrame(columns = ['Candidate_Name','Content_CV'])
i=0
while i < len(onlyfiles):
    file=onlyfiles[i]
    base = os.path.basename(file)  #Test_Phoebe Buffay.pdf
    filename = os.path.splitext(base)[0]  #Test_Phoebe Buffay
    dat=extract_text(file)
    data = [{'Candidate_Name':filename, 'Content_CV':dat}]
    df_Candidates=df_Candidates.append(data, ignore_index=True)
    i+=1

df_Candidates = df_Candidates.sort_values('Candidate_Name', ascending=True)
df_Candidates

,Candidate_Name,Content_CV
6,Data_Scientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...
2,Data_Scientist_Rahul_Malik,\nRAHUL MALIK\nNLP Data Scientist\nCONTACT WOR...
4,HCM_Federico_Calonge,\n ...
7,HCM_Robert_Smith,\nSap Hcm Consultant Phone: (123) 456 78 99\nE...
0,ML_Engineer_Bradly_Johnston,"\nBradly Johnston\n435 Kasey Vista, Detroit\n+..."
1,ML_Engineer_Jonathon_Price,"\nJonathon Price \n4587 Terry Groves, Boston\n..."
8,Security_Specialist_Ahmed Wayne,"\nAhmed Wayne\nAddress: Abu Dhabi, UAE\nNation..."
3,Security_Specialist_Denis Banik,\nDenis Banik\nEmail address: hello@kickresume...
5,Web_Dev_Alec_Dionisio,"\nChestertown, MD 4107083942\nhi@alecdionis.io..."
9,Web_Dev_Karen_Higgins,\nKaren Higgins \n We b Developer \n \n...


**Limpieza De los CVs**:
1. Limpieza "general": Pasar todo a minúscula, eliminar signos de puntuación, espacios en blanco.
2. Limpieza de stop words.
3. Steaming para unir palabras: computer_science, machine_learning.
4. Lematización. (VER)
5. Limpieza en txt de datos personales (mails, telefonos).
6. Limpieza de palabras innecesarias en archivo txt (Page, meses, años, etc).

**Limpieza de los Puestos**:
1. Limpieza "general": Pasar todo a minúscula, eliminar signos de puntuación, espacios en blanco.
2. Limpieza de stop words.
3. Steaming para unir palabras: computer_science, machine_learning.
4. Lematización. (VER)
5. Limpieza de palabras innecesarias en archivo txt (VER: por ej. mails que dejan las empresas, telefonos, etc.).
6. Luego, para la obtención de Keywords (VER si es necesario): --> Por el problema que me tomaba las keywords como "you", "I" y demás..., podría hacer un txt con lenguajes y skills por "defecto" y me fijo si están en las keywords... sino no las pongo.


### Funciones para limpiar datos en columnas de un DF

In [5]:
import regex as re #Usado en la función remove_punctuation_and_special_characters.
from nltk.stem import WordNetLemmatizer   #Usado para lematización.
nltk.download('wordnet')
from nltk.tokenize import word_tokenize   #Usado para tokenizar.

#Definimos nuestras stop words desde nltk:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
                
def lower_text(DF,clean_column):
    DF[clean_column] = DF[clean_column].apply(lambda x: x.lower() if isinstance(x,str) else x)
    
def remove_stop_words(DF,clean_column):
    DF[clean_column] = DF[clean_column].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

def remove_punctuation_and_special_characters(DF,clean_column):
    DF[clean_column] = DF[clean_column].apply(lambda x: re.sub('[^\w-]|(\d+)',' ', x) if isinstance(x,str) else x)
                                                                #Macheamos cualquier non-word excepto el -; y tambien macheamos los números (\d+)
    #Macheos:
        #'\W+'                  --> Cualquier non-word.
        #'[()!@#$/,;.:"]'       --> todos esos carácteres especiales que pongo ahi. 
        #'\w+[-]\w+'            --> machea a las palabras non-algo.
        #'[^\w-+]'              --> Machea todas las non-words excepto el guion medio y el +.
        #'\C\+\+'               --> Machea C++.
        #[^\w-+]|(\C\+\+|team)  --> Machea todas las non-words excepto el guion medio y el +. Y tambien machea C++ y la palabra "team"
        #[^\w-+]|(\C\+\+|\d+)   --> Machea todas las non-words excepto el guion medio y el +. Y tambien machea C++ y los números (\d+).
        
    #Para m.sc' --> reemplazar el . por espacio cuando antes y despues del punto esta todo "junto"... para diferenciarlo del punto de una oración. 
    
#Tokenizar es el proceso de parsear Strings de texto en diferentes secciones ("tokens"). Debemos tokenizar teniendo en cuenta qué hacer con las comas, con los espacios en blanco, 
#con los "-", etc. Para esto usamos la función "word_tokenize" que se encargará de esta tarea. 
#Esta funcion NO divide las palabras que estan con un "-"... de esta manera la palabra "scale-up" queda así y NO se divide.
#Tambien tendremos la palabra con una única letra ("c", del lenguaje c) macheado únicamente a un token "c". 
def tokenize_and_lemmatization(text_column):
    tokens = word_tokenize(text_column) #Tokenizamos.
    wordnet_lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [wordnet_lemmatizer.lemmatize(tok).lower() for tok in tokens] # Stem words.
    return list(lemmatized_tokens)
    
def cleaning_DF(DF,column_to_clean):
    clean_column='clean_'+column_to_clean
    DF[clean_column]=DF[column_to_clean] #Copiamos el contenido de 'column_to_clean' en 'clean_column' para utilizarla en las funciones posteriores.
    lower_text(DF,clean_column)
    remove_punctuation_and_special_characters(DF,clean_column)
    remove_stop_words(DF,clean_column)
     
def tokenize_and_lemmatize(DF,column_to_clean):
    clean_column='clean_'+column_to_clean
    tokens_column='tokens_'+column_to_clean
    DF[tokens_column] = DF[clean_column].apply(tokenize_and_lemmatization)

[nltk_data] Downloading package wordnet to /home/fedricio/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#### Limpieza de datos del DF de Jobs.

In [6]:
cleaning_DF(df_Jobs,'Job_Description')
tokenize_and_lemmatize(df_Jobs,'Job_Description')


#Tomamos el job_description de la posición 5 como ejemplo (Machine Learning Engineer 2'):

print(df_Jobs['Job_Description'].iloc[5])  
#non-expert  debería mantenerse.
#deberia juntar el Computer Science
print("#######################################################")
print(df_Jobs['clean_Job_Description'].iloc[5]) 
print("#######################################################")
print(df_Jobs['tokens_Job_Description'].iloc[5]) 
print("#######################################################")

Collaborate with a multidisciplinary team to gain insight into complex biochemical systems.
Design computational models to study various interactions such as interactions between genomes, proteins, and binding sites
Extract various features from the computational models and communicate the results back to the team.
Predict the behaviour of new protein structures on certain binding sites using the computational models.
Work with the team of software engineers to embed your models into production.
Perform other related duties in keeping with the purpose and accountabilities of the job.

M.Sc. or Ph.D. in Engineering/Computer Science, or an equivalent combination of experience and knowledge.
2+ years' experience applying machine learning and deep learning concepts to real-world problems.
Solid programming skills with a focus on writing clean/maintainable code, with 2+ years of experience in Python (preferred), Java, or C++ programming.
Good knowledge of machine learning libraries (Tensorf

In [7]:
df_Jobs

,Job_Title,Job_Description,clean_Job_Description,tokens_Job_Description
2,Data Scientist,"Master’s degree or above in a STEM field, incl...",master degree stem field including limited com...,"[master, degree, stem, field, including, limit..."
3,Data Scientist 2,"\nReporting to the Director, Data & Analytics,...",reporting director data analytics senior data ...,"[reporting, director, data, analytics, senior,..."
6,HCM Consultant,\nThe Oracle Cloud HCM Absence Consultant will...,oracle cloud hcm absence consultant responsibl...,"[oracle, cloud, hcm, absence, consultant, resp..."
7,HCM Consultant 2,4+ years of experience in PeopleSoft or Oracle...,years experience peoplesoft oracle ebs impleme...,"[year, experience, peoplesoft, oracle, eb, imp..."
0,Machine Learning Engineer,Leveraging the latest machine and deep learnin...,leveraging latest machine deep learning techni...,"[leveraging, latest, machine, deep, learning, ..."
1,Machine Learning Engineer 2,Collaborate with a multidisciplinary team to g...,collaborate multidisciplinary team gain insigh...,"[collaborate, multidisciplinary, team, gain, i..."
8,Security Specialist,Work in a fast-paced environment that combine ...,work fast-paced environment combine technical ...,"[work, fast-paced, environment, combine, techn..."
9,Security Specialist 2,\n Handling incoming requests for assistanc...,handling incoming requests assistance business...,"[handling, incoming, request, assistance, busi..."
4,Web Developer Full Stack,\n\n Graduate Degree in Information Technol...,graduate degree information technology similar...,"[graduate, degree, information, technology, si..."
5,Web Developer Full Stack 2,\n· Enter existing website codebases and exten...,enter existing website codebases extend functi...,"[enter, existing, website, codebases, extend, ..."


#### Limpieza de datos del DF de Candidatos.

In [8]:
cleaning_DF(df_Candidates,'Content_CV')
tokenize_and_lemmatize(df_Candidates,'Content_CV')

df_Candidates

,Candidate_Name,Content_CV,clean_Content_CV,tokens_Content_CV
6,Data_Scientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,karla lewis data scientist contact work experi...,"[karla, lewis, data, scientist, contact, work,..."
2,Data_Scientist_Rahul_Malik,\nRAHUL MALIK\nNLP Data Scientist\nCONTACT WOR...,rahul malik nlp data scientist contact work ex...,"[rahul, malik, nlp, data, scientist, contact, ..."
4,HCM_Federico_Calonge,\n ...,calonge federico matías hcm technical consulta...,"[calonge, federico, matías, hcm, technical, co..."
7,HCM_Robert_Smith,\nSap Hcm Consultant Phone: (123) 456 78 99\nE...,sap hcm consultant phone email info qwikresume...,"[sap, hcm, consultant, phone, email, info, qwi..."
0,ML_Engineer_Bradly_Johnston,"\nBradly Johnston\n435 Kasey Vista, Detroit\n+...",bradly johnston kasey vista detroit work exper...,"[bradly, johnston, kasey, vista, detroit, work..."
1,ML_Engineer_Jonathon_Price,"\nJonathon Price \n4587 Terry Groves, Boston\n...",jonathon price terry groves boston experience ...,"[jonathon, price, terry, grove, boston, experi..."
8,Security_Specialist_Ahmed Wayne,"\nAhmed Wayne\nAddress: Abu Dhabi, UAE\nNation...",ahmed wayne address abu dhabi uae nationality ...,"[ahmed, wayne, address, abu, dhabi, uae, natio..."
3,Security_Specialist_Denis Banik,\nDenis Banik\nEmail address: hello@kickresume...,denis banik email address hello kickresume com...,"[denis, banik, email, address, hello, kickresu..."
5,Web_Dev_Alec_Dionisio,"\nChestertown, MD 4107083942\nhi@alecdionis.io...",chestertown md hi alecdionis io alecdionis io ...,"[chestertown, md, hi, alecdionis, io, alecdion..."
9,Web_Dev_Karen_Higgins,\nKaren Higgins \n We b Developer \n \n...,karen higgins b developer areas expertise pers...,"[karen, higgins, b, developer, area, expertise..."


In [9]:
#Vemos los tokens generados para el candidato HCM_Federico_Calonge:
df_Candidates.iloc[2]['tokens_Content_CV']

['calonge',
 'federico',
 'matías',
 'hcm',
 'technical',
 'consultant',
 'federico',
 'working',
 'oracle',
 'tool',
 'since',
 'october',
 'march',
 'participated',
 'erp',
 'cloud',
 'project',
 'performing',
 'reporting',
 'task',
 'working',
 'module',
 'ap',
 'ar',
 'gl',
 'since',
 'march',
 'participating',
 'hcm',
 'cloud',
 'project',
 'performing',
 'reporting',
 'extraction',
 'integration',
 'task',
 'working',
 'module',
 'core',
 'recruitment',
 'since',
 'october',
 'member',
 'artificial',
 'intelligence',
 'ai',
 'committee',
 'oracle',
 'strong',
 'interest',
 'data',
 'science',
 'machine',
 'learning',
 'last',
 'year',
 'computer',
 'engineering',
 'degree',
 'studying',
 'th',
 'year',
 'english',
 'personal',
 'academic',
 'programming',
 'project',
 'http',
 'github',
 'com',
 'federicocalonge',
 'skill',
 'summary',
 'oracle',
 'hcm',
 'cloud',
 'core',
 'recruitment',
 'oracle',
 'erp',
 'cloud',
 'account',
 'receivable',
 'ar',
 'account',
 'payable',
 'ap'

### AGREGADO: Funciones para tokenizar textos de una columna de un DF

In [10]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
import string
#Esto de arriba se descarga LOCALMENTE en home/user/nltk_data.

def tokenize(text):
    #Convierte el texto recibido como entrada en una lista de tokens (palabras).
    #Dentro pasamos todo a minusculas, se eliminan signos de puntuación, espacios en blanco y se eliminan las stop words.
        #input  --> text: el documento como un STRING. 
        #output --> texts_tokens: cada palabra del texto en una LISTA de string. 
    texts_tokens=[]
    for line in text:
        tokens=word_tokenize(line)
        tok=[w.lower() for w in tokens]
        table=str.maketrans('','',string.punctuation)
        strpp=[w.translate(table) for w in tok]
        words=[word for word in strpp if word.isalpha()]
        stop_words=set(stopwords.words('english'))
        words=[w for w in words if not w in stop_words]
        texts_tokens.append(words)
    return texts_tokens

def text_to_vector(text):
    #Convierte el texto del documento en una "term matrix" donde todas las palabras estan listadas junto a 
    #la frecuencia en que aparecen en el texto.
        #input  --> text: el documento como una LISTA de string. 
        #output --> Term matrix: una MATRIZ con cada palabra del documento junto a su frecuencia. 
        
    texts_tokens = tokenize(text)
    #print(texts_tokens)
    return Counter(texts_tokens[0]) #[0] porque es una lista dentro de otra lista.

[nltk_data] Downloading package punkt to /home/fedricio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/fedricio/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# ---------------------------------------------------------------

### PRUEBAS CON TF IDF +Cosine:

In [11]:
#Nos quedarán 10 x 10 = 100 filas.
df_Jobs_and_Candidates = pd.merge(df_Candidates.assign(A=1), df_Jobs.assign(A=1), on='A').drop('A', 1)
df_Jobs_and_Candidates

,Candidate_Name,Content_CV,clean_Content_CV,tokens_Content_CV,Job_Title,Job_Description,clean_Job_Description,tokens_Job_Description
0,Data_Scientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,karla lewis data scientist contact work experi...,"[karla, lewis, data, scientist, contact, work,...",Data Scientist,"Master’s degree or above in a STEM field, incl...",master degree stem field including limited com...,"[master, degree, stem, field, including, limit..."
1,Data_Scientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,karla lewis data scientist contact work experi...,"[karla, lewis, data, scientist, contact, work,...",Data Scientist 2,"\nReporting to the Director, Data & Analytics,...",reporting director data analytics senior data ...,"[reporting, director, data, analytics, senior,..."
2,Data_Scientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,karla lewis data scientist contact work experi...,"[karla, lewis, data, scientist, contact, work,...",HCM Consultant,\nThe Oracle Cloud HCM Absence Consultant will...,oracle cloud hcm absence consultant responsibl...,"[oracle, cloud, hcm, absence, consultant, resp..."
3,Data_Scientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,karla lewis data scientist contact work experi...,"[karla, lewis, data, scientist, contact, work,...",HCM Consultant 2,4+ years of experience in PeopleSoft or Oracle...,years experience peoplesoft oracle ebs impleme...,"[year, experience, peoplesoft, oracle, eb, imp..."
4,Data_Scientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,karla lewis data scientist contact work experi...,"[karla, lewis, data, scientist, contact, work,...",Machine Learning Engineer,Leveraging the latest machine and deep learnin...,leveraging latest machine deep learning techni...,"[leveraging, latest, machine, deep, learning, ..."
...,...,...,...,...,...,...,...,...
95,Web_Dev_Karen_Higgins,\nKaren Higgins \n We b Developer \n \n...,karen higgins b developer areas expertise pers...,"[karen, higgins, b, developer, area, expertise...",Machine Learning Engineer 2,Collaborate with a multidisciplinary team to g...,collaborate multidisciplinary team gain insigh...,"[collaborate, multidisciplinary, team, gain, i..."
96,Web_Dev_Karen_Higgins,\nKaren Higgins \n We b Developer \n \n...,karen higgins b developer areas expertise pers...,"[karen, higgins, b, developer, area, expertise...",Security Specialist,Work in a fast-paced environment that combine ...,work fast-paced environment combine technical ...,"[work, fast-paced, environment, combine, techn..."
97,Web_Dev_Karen_Higgins,\nKaren Higgins \n We b Developer \n \n...,karen higgins b developer areas expertise pers...,"[karen, higgins, b, developer, area, expertise...",Security Specialist 2,\n Handling incoming requests for assistanc...,handling incoming requests assistance business...,"[handling, incoming, request, assistance, busi..."
98,Web_Dev_Karen_Higgins,\nKaren Higgins \n We b Developer \n \n...,karen higgins b developer areas expertise pers...,"[karen, higgins, b, developer, area, expertise...",Web Developer Full Stack,\n\n Graduate Degree in Information Technol...,graduate degree information technology similar...,"[graduate, degree, information, technology, si..."


In [12]:
#https://www.py4u.net/discuss/188191
    
#Calculate cosine similarity using tf-idf vectors, between search queries and matched documents.
#Acá mi search query es la columna clean_Content_CV y el matched documents son los clean_Job_Description.
#Cuando hacemos un transform estamos pasandole la query para que se fije los TF-IDF que tiene ya entrenados
#del modelo (cuando hicimos fit ya lo entrenamos... y lo hicimos con los cvs y los job descriptions). 

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import paired_cosine_distances as pcd

# Initialize an instance of tf-idf Vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Generate the tf-idf vectors for the corpus
tfidf_vectorizer.fit(df_Jobs_and_Candidates['clean_Content_CV'] + " " + df_Jobs_and_Candidates['clean_Job_Description'])

A = tfidf_vectorizer.transform(df_Jobs_and_Candidates['clean_Content_CV'])
B = tfidf_vectorizer.transform(df_Jobs_and_Candidates['clean_Job_Description'])

# compute and the cosine similarity:
cosine = 1 - pcd(A, B)

df_Jobs_and_Candidates['tfidf_cosine'] = cosine

df_Jobs_and_Candidates

,Candidate_Name,Content_CV,clean_Content_CV,tokens_Content_CV,Job_Title,Job_Description,clean_Job_Description,tokens_Job_Description,tfidf_cosine
0,Data_Scientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,karla lewis data scientist contact work experi...,"[karla, lewis, data, scientist, contact, work,...",Data Scientist,"Master’s degree or above in a STEM field, incl...",master degree stem field including limited com...,"[master, degree, stem, field, including, limit...",0.058337
1,Data_Scientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,karla lewis data scientist contact work experi...,"[karla, lewis, data, scientist, contact, work,...",Data Scientist 2,"\nReporting to the Director, Data & Analytics,...",reporting director data analytics senior data ...,"[reporting, director, data, analytics, senior,...",0.099616
2,Data_Scientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,karla lewis data scientist contact work experi...,"[karla, lewis, data, scientist, contact, work,...",HCM Consultant,\nThe Oracle Cloud HCM Absence Consultant will...,oracle cloud hcm absence consultant responsibl...,"[oracle, cloud, hcm, absence, consultant, resp...",0.028673
3,Data_Scientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,karla lewis data scientist contact work experi...,"[karla, lewis, data, scientist, contact, work,...",HCM Consultant 2,4+ years of experience in PeopleSoft or Oracle...,years experience peoplesoft oracle ebs impleme...,"[year, experience, peoplesoft, oracle, eb, imp...",0.020506
4,Data_Scientist_Karla_Lewis,\nKARLA LEWIS\nData Scientist\nCONTACT WORK EX...,karla lewis data scientist contact work experi...,"[karla, lewis, data, scientist, contact, work,...",Machine Learning Engineer,Leveraging the latest machine and deep learnin...,leveraging latest machine deep learning techni...,"[leveraging, latest, machine, deep, learning, ...",0.051409
...,...,...,...,...,...,...,...,...,...
95,Web_Dev_Karen_Higgins,\nKaren Higgins \n We b Developer \n \n...,karen higgins b developer areas expertise pers...,"[karen, higgins, b, developer, area, expertise...",Machine Learning Engineer 2,Collaborate with a multidisciplinary team to g...,collaborate multidisciplinary team gain insigh...,"[collaborate, multidisciplinary, team, gain, i...",0.074330
96,Web_Dev_Karen_Higgins,\nKaren Higgins \n We b Developer \n \n...,karen higgins b developer areas expertise pers...,"[karen, higgins, b, developer, area, expertise...",Security Specialist,Work in a fast-paced environment that combine ...,work fast-paced environment combine technical ...,"[work, fast-paced, environment, combine, techn...",0.081982
97,Web_Dev_Karen_Higgins,\nKaren Higgins \n We b Developer \n \n...,karen higgins b developer areas expertise pers...,"[karen, higgins, b, developer, area, expertise...",Security Specialist 2,\n Handling incoming requests for assistanc...,handling incoming requests assistance business...,"[handling, incoming, request, assistance, busi...",0.051793
98,Web_Dev_Karen_Higgins,\nKaren Higgins \n We b Developer \n \n...,karen higgins b developer areas expertise pers...,"[karen, higgins, b, developer, area, expertise...",Web Developer Full Stack,\n\n Graduate Degree in Information Technol...,graduate degree information technology similar...,"[graduate, degree, information, technology, si...",0.100100


In [14]:
#SI se agrega un Candidato nuevo, se agrega al DF y hago que me devuelva la columna en la que se insertó. 
#Y con esa columna hago el cálculo que hice arriba.. por ejemplo ahora para la de indice 96:
A = tfidf_vectorizer.transform([df_Jobs_and_Candidates['clean_Content_CV'][96]])
B = tfidf_vectorizer.transform([df_Jobs_and_Candidates['clean_Job_Description'][96]])

cosine = 1 - pcd(A, B)
cosine
#df_Jobs_and_Candidates['tfidf_cosine'] = cosine

array([0.08198229])